<a href="https://colab.research.google.com/github/AMRUTA-BANDHU-CHAUDHURY/AI-ML-DL-REPO/blob/master/BayesianNetworks_Asia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Bayesian Networks Code Demo - Asia Usecase 

#### A Bayesian network is a directed acyclic graphical model whose nodes represent a set of random variables and their conditional dependencies via a directed acyclic graph (DAG). We shall look at a Bayesian network called the 'asia network' develped by Lauritzen and Speigelhalter(1988) which represent the probability of developing a lung cancer or bronchitis. Pgmpy is the python library used for demonstration

In [0]:
## setting the current directory
import os
path = "C:/Users/usha/Desktop/BangPypers/asia"
os.chdir(path)


#### BIFReader is a pgmpy package used for reading network file in bif format



In [0]:
## reading the file 'asia.bif' using BIFReader
from pgmpy.readwrite import BIFReader
reader = BIFReader('asia.bif')
asia_model = reader.get_model()

#### nodes of the graph are retrieved using the nodes() method

In [0]:
## nodes of the asia model
asia_model.nodes()


NodeView(('asia', 'tub', 'smoke', 'lung', 'bronc', 'either', 'xray', 'dysp'))

#### Edges of the graph is retrieved using edges() method 

In [0]:
## edges of the asia model
asia_model.edges()


OutEdgeView([('asia', 'tub'), ('tub', 'either'), ('smoke', 'lung'), ('smoke', 'bronc'), ('lung', 'either'), ('bronc', 'dysp'), ('either', 'xray'), ('either', 'dysp')])

#### In Bayesian Networks, the relationship between nodes are specified by CPDs. get_cpds() adds table to BIF model

In [0]:
## the cpds of asia model
asia_model.get_cpds()

[<TabularCPD representing P(asia:2) at 0x27024009ef0>,
 <TabularCPD representing P(bronc:2 | smoke:2) at 0x270241152b0>,
 <TabularCPD representing P(dysp:2 | bronc:2, either:2) at 0x270241151d0>,
 <TabularCPD representing P(either:2 | lung:2, tub:2) at 0x27024115320>,
 <TabularCPD representing P(lung:2 | smoke:2) at 0x27026103f60>,
 <TabularCPD representing P(smoke:2) at 0x27026103f98>,
 <TabularCPD representing P(tub:2 | asia:2) at 0x27026103cc0>,
 <TabularCPD representing P(xray:2 | either:2) at 0x27026103be0>]

#### The cpds of all nodes are printed by iterating through asia_model.get_cpds()

In [0]:
# Iterate over asia_model.get_cpds()
for cpd in asia_model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)

CPD of asia:
╒════════╤══════╕
│ asia_0 │ 0.01 │
├────────┼──────┤
│ asia_1 │ 0.99 │
╘════════╧══════╛
CPD of bronc:
╒═════════╤═════════╤═════════╕
│ smoke   │ smoke_0 │ smoke_1 │
├─────────┼─────────┼─────────┤
│ bronc_0 │ 0.6     │ 0.3     │
├─────────┼─────────┼─────────┤
│ bronc_1 │ 0.4     │ 0.7     │
╘═════════╧═════════╧═════════╛
CPD of dysp:
╒════════╤══════════╤══════════╤══════════╤══════════╕
│ bronc  │ bronc_0  │ bronc_0  │ bronc_1  │ bronc_1  │
├────────┼──────────┼──────────┼──────────┼──────────┤
│ either │ either_0 │ either_1 │ either_0 │ either_1 │
├────────┼──────────┼──────────┼──────────┼──────────┤
│ dysp_0 │ 0.9      │ 0.7      │ 0.8      │ 0.1      │
├────────┼──────────┼──────────┼──────────┼──────────┤
│ dysp_1 │ 0.1      │ 0.3      │ 0.2      │ 0.9      │
╘════════╧══════════╧══════════╧══════════╧══════════╛
CPD of either:
╒══════════╤════════╤════════╤════════╤════════╕
│ lung     │ lung_0 │ lung_0 │ lung_1 │ lung_1 │
├──────────┼────────┼────────┼────────

#### Exact inference using Variable Elimination


In [0]:
# Doing exact inference using Variable Elimination
from pgmpy.inference import VariableElimination
asia_infer = VariableElimination(asia_model)

# Computing the probability of bronc given smoke.
a = asia_infer.query(variables=['bronc'], evidence={'smoke': 0})
print(a['bronc'])

╒═════════╤══════════════╕
│ bronc   │   phi(bronc) │
╞═════════╪══════════════╡
│ bronc_0 │       0.6000 │
├─────────┼──────────────┤
│ bronc_1 │       0.4000 │
╘═════════╧══════════════╛


#### Deriving inference when evidence is not observed

In [0]:
# Computing the probability of xray when there is no evidence
b = asia_infer.query(variables=['xray'], evidence={})
print(b['xray'])

## Computing the probability of smoke when there is no evidence
c = asia_infer.query(variables=['smoke'], evidence={})
print(c['smoke'])


╒════════╤═════════════╕
│ xray   │   phi(xray) │
╞════════╪═════════════╡
│ xray_0 │      0.1103 │
├────────┼─────────────┤
│ xray_1 │      0.8897 │
╘════════╧═════════════╛
╒═════════╤══════════════╕
│ smoke   │   phi(smoke) │
╞═════════╪══════════════╡
│ smoke_0 │       0.5000 │
├─────────┼──────────────┤
│ smoke_1 │       0.5000 │
╘═════════╧══════════════╛


#### Deriving inference when evidence is observed

In [0]:
## Computing the probability of lung given the probability of observing smoke is zero
c = asia_infer.query(variables=[('lung')], evidence={'smoke': 0})
print(c['lung'])

## Computing the probability of dysp given the probability of observing smoke is zero
d = asia_infer.query(variables=['dysp'], evidence={'smoke' : 0})
print(d['dysp'])


╒════════╤═════════════╕
│ lung   │   phi(lung) │
╞════════╪═════════════╡
│ lung_0 │      0.1000 │
├────────┼─────────────┤
│ lung_1 │      0.9000 │
╘════════╧═════════════╛
╒════════╤═════════════╕
│ dysp   │   phi(dysp) │
╞════════╪═════════════╡
│ dysp_0 │      0.5037 │
├────────┼─────────────┤
│ dysp_1 │      0.4963 │
╘════════╧═════════════╛


In [0]:
## Computing the probability of lung given the probability of observing smoke is one
e = asia_infer.query(variables=['lung'], evidence={'smoke': 1})
print(e['lung'])

## Computing the probability of dysp given the probability of observing smoke is one
f = asia_infer.query(variables=['dysp'], evidence={'smoke': 1})
print(f['dysp'])

╒════════╤═════════════╕
│ lung   │   phi(lung) │
╞════════╪═════════════╡
│ lung_0 │      0.0100 │
├────────┼─────────────┤
│ lung_1 │      0.9900 │
╘════════╧═════════════╛
╒════════╤═════════════╕
│ dysp   │   phi(dysp) │
╞════════╪═════════════╡
│ dysp_0 │      0.2912 │
├────────┼─────────────┤
│ dysp_1 │      0.7088 │
╘════════╧═════════════╛
